### Load Model

In [ ]:

import sys
sys.path.insert(0, '../')
import aux
from aux import *
#%load_ext autoreload
#%autoreload 2
##Clear Memory 
tf.reset_default_graph()
tf.keras.backend.clear_session()
gc.collect()
##


NUM_GPU = len(get_available_gpus())
if(NUM_GPU>0) :
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

print(tf.__version__)
print(keras.__version__)
#tpu_grpc_url = TPUClusterResolver(tpu=['edv-tpu2'] , zone='us-central1-a').get_master()




### Load the Model in a separate graph here as there are two models in this figure.
fitness_function_graph = tf.Graph()
with fitness_function_graph.as_default():
    model_conditions='Glu'
    model, scaler,batch_size = load_model(model_conditions)
    
plotting_alpha=0.2

### Load Data

<li>Y_pred contains the predictions
<li>X is the format of one-hot encoded input expected

In [ ]:
def read_hq_testdata(filename) :

    with open(filename) as f:
        reader = csv.reader(f, delimiter="\t")
        d = list(reader)

    sequences = [di[0] for di in d]

    for i in tqdm(range(0,len(sequences))) : 
        if (len(sequences[i]) > 110) :
            sequences[i] = sequences[i][-110:]
        if (len(sequences[i]) < 110) : 
            while (len(sequences[i]) < 110) :
                sequences[i] = 'N'+sequences[i]



    A_onehot = np.array([1,0,0,0] ,  dtype=np.bool)
    C_onehot = np.array([0,1,0,0] ,  dtype=np.bool)
    G_onehot = np.array([0,0,1,0] ,  dtype=np.bool)
    T_onehot = np.array([0,0,0,1] ,  dtype=np.bool)
    N_onehot = np.array([0,0,0,0] ,  dtype=np.bool)

    mapper = {'A':A_onehot,'C':C_onehot,'G':G_onehot,'T':T_onehot,'N':N_onehot}
    worddim = len(mapper['A'])
    seqdata = np.asarray(sequences)
    seqdata_transformed = seq2feature(seqdata)
    print(seqdata_transformed.shape)


    expressions = [di[1] for di in d]
    expdata = np.asarray(expressions)
    expdata = expdata.astype('float')  

    return np.squeeze(seqdata_transformed),expdata

X,Y = read_hq_testdata(os.path.join('..','..','data','Glu','HQ_testdata.txt'))
Y = [float(x) for x in Y]
Y_pred = evaluate_model(X, model, scaler, batch_size , fitness_function_graph)



#plt.scatter(Y_pred,Y)

plt.xlabel('Predicted')
plt.ylabel('Measured')
#print('MSE',sklearn.metrics.mean_squared_error(Y_pred,Y))




### Evaluate Model 
<li> Here, we have the measured expression values Y corresponding to the sequences X

In [ ]:
fig=plt.figure(figsize=(4,4), dpi= 200, facecolor='w', edgecolor='k')

sns.regplot(Y_pred,Y , s=5 , linewidth=0.25)
plt.title(scipy.stats.spearmanr(Y_pred,Y)[0])
plt.xlabel('Predicted')
plt.ylabel('Measured')
ax = plt.gca()
plt.xticks([])
plt.yticks([])
ax.autoscale(enable=True, axis='x', tight=True)
ax.autoscale(enable=True, axis='y', tight=True)

